In [2]:
from typing import LiteralString
import os
import duckdb

def process(sql: LiteralString, file: str) -> None:
    table = duckdb.sql(sql)
    table.to_parquet(f"./tmp/{file}.parquet")
    print(f"{file} - {int(os.path.getsize(f"./tmp/{file}.parquet") / (1024 * 1024))}")
    table.show()

In [18]:
process("""
SELECT
    region,
    date,
    url[length('https://open.spotify.com/track/')+1:] as track_id,
    streams
FROM '../../data/charts.csv'
WHERE chart = 'top200'
""", "charts_fmt")

charts_fmt - 381
┌───────────┬────────────┬────────────────────────┬─────────┐
│  region   │    date    │        track_id        │ streams │
│  varchar  │    date    │        varchar         │  int64  │
├───────────┼────────────┼────────────────────────┼─────────┤
│ Argentina │ 2017-01-01 │ 6mICuAdrwEjh6Y6lroV2Kg │  253019 │
│ Argentina │ 2017-01-01 │ 7DM4BPaS7uofFul3ywMe46 │  223988 │
│ Argentina │ 2017-01-01 │ 3AEZUABDXNtecAOSC1qTfo │  210943 │
│ Argentina │ 2017-01-01 │ 6rQSrBHf7HlZjtcMZ4S4bO │  173865 │
│ Argentina │ 2017-01-01 │ 58IL315gMSTD37DOZPJ2hf │  153956 │
│ Argentina │ 2017-01-01 │ 5J1c3M4EldCfNxXwrwt8mT │  151140 │
│ Argentina │ 2017-01-01 │ 1MpKZi1zTXpERKwxmOu1PH │  148369 │
│ Argentina │ 2017-01-01 │ 3QwBODjSEzelZyVjxPOHdq │  143004 │
│ Argentina │ 2017-01-01 │ 0sXvAOmXgjR2QUqLK1MltU │  126389 │
│ Argentina │ 2017-01-01 │ 20ZAJdsKB5IGbGj4ilRt2o │  112012 │
│   ·       │     ·      │           ·            │     ·   │
│   ·       │     ·      │           ·            │  

In [38]:
process("""
SELECT
    region,
    date,
    SUM(streams) as streams
FROM './tmp/charts_fmt.parquet'
GROUP BY region, date
ORDER BY region, date
""", "charts_daily_sum")

charts_daily_sum - 0
┌───────────┬────────────┬──────────┐
│  region   │    date    │ streams  │
│  varchar  │    date    │  int128  │
├───────────┼────────────┼──────────┤
│ Argentina │ 2017-01-01 │  7888872 │
│ Argentina │ 2017-01-02 │  6010041 │
│ Argentina │ 2017-01-03 │  5921907 │
│ Argentina │ 2017-01-04 │  6019573 │
│ Argentina │ 2017-01-05 │  6223646 │
│ Argentina │ 2017-01-06 │  6485179 │
│ Argentina │ 2017-01-07 │  6840811 │
│ Argentina │ 2017-01-08 │  5691182 │
│ Argentina │ 2017-01-09 │  5880184 │
│ Argentina │ 2017-01-10 │  5896533 │
│   ·       │     ·      │     ·    │
│   ·       │     ·      │     ·    │
│   ·       │     ·      │     ·    │
│ Brazil    │ 2019-07-13 │ 33582302 │
│ Brazil    │ 2019-07-14 │ 27615671 │
│ Brazil    │ 2019-07-15 │ 25856711 │
│ Brazil    │ 2019-07-16 │ 26449349 │
│ Brazil    │ 2019-07-17 │ 26636713 │
│ Brazil    │ 2019-07-18 │ 27154103 │
│ Brazil    │ 2019-07-19 │ 30321659 │
│ Brazil    │ 2019-07-20 │ 32104543 │
│ Brazil    │ 2019-07-21 │ 27

In [11]:
process("""
SELECT
    region,
    YEAR(date) as year,
    AVG(streams) as stream_avg,
    STDDEV(streams) as stream_dev
FROM './tmp/charts_daily_sum.parquet'
GROUP BY region, year
ORDER BY region, year
""", "charts_yearly_stats")

charts_yearly_stats - 0
┌───────────────┬───────┬────────────────────┬────────────────────┐
│    region     │ year  │     stream_avg     │     stream_dev     │
│    varchar    │ int64 │       double       │       double       │
├───────────────┼───────┼────────────────────┼────────────────────┤
│ Argentina     │  2017 │  7424229.113259668 │ 1034041.7027895374 │
│ Argentina     │  2018 │  8642794.898630137 │  971236.3782346837 │
│ Argentina     │  2019 │ 10852934.652054794 │ 1505170.5526055752 │
│ Argentina     │  2020 │ 11974701.893442623 │ 2195595.9299331396 │
│ Argentina     │  2021 │  14691098.39010989 │ 1835217.3342268008 │
│ Australia     │  2017 │  9070440.314917127 │ 1217620.9157923514 │
│ Australia     │  2018 │ 11125632.709589042 │ 1184823.8273182358 │
│ Australia     │  2019 │ 11930231.084931508 │  1209072.960792116 │
│ Australia     │  2020 │ 11986660.775956284 │ 1581691.8264637932 │
│ Australia     │  2021 │ 11450993.815864023 │  724796.2088324842 │
│     ·         │    ·  

In [17]:
process("""
SELECT
    region,
    date,
    CASE
        WHEN stream_std < -1.5 THEN 'VERY LOW'
        WHEN stream_std < -0.5 THEN 'LOW'
        WHEN stream_std > 1.5 THEN 'VERY HIGH'
        WHEN stream_std > 0.5 THEN 'HIGH'
        ELSE 'AVERAGE'
    END AS popularity
FROM (
    SELECT
        d.region,
        d.date,
        (d.streams - y.stream_avg) / y.stream_dev as stream_std
    FROM './tmp/charts_daily_sum.parquet' AS d
    JOIN './tmp/charts_yearly_stats.parquet' AS y
        ON YEAR(d.date) = y.year AND d.region = y.region
)
""", "charts_daily_popularity")

charts_daily_popularity - 0
┌───────────┬────────────┬────────────┐
│  region   │    date    │ popularity │
│  varchar  │    date    │  varchar   │
├───────────┼────────────┼────────────┤
│ Argentina │ 2017-01-01 │ AVERAGE    │
│ Argentina │ 2017-01-02 │ LOW        │
│ Argentina │ 2017-01-03 │ LOW        │
│ Argentina │ 2017-01-04 │ LOW        │
│ Argentina │ 2017-01-05 │ LOW        │
│ Argentina │ 2017-01-06 │ LOW        │
│ Argentina │ 2017-01-07 │ LOW        │
│ Argentina │ 2017-01-08 │ VERY LOW   │
│ Argentina │ 2017-01-09 │ LOW        │
│ Argentina │ 2017-01-10 │ LOW        │
│   ·       │     ·      │  ·         │
│   ·       │     ·      │  ·         │
│   ·       │     ·      │  ·         │
│ Brazil    │ 2018-04-26 │ AVERAGE    │
│ Brazil    │ 2018-04-27 │ HIGH       │
│ Brazil    │ 2018-04-28 │ VERY HIGH  │
│ Brazil    │ 2018-04-29 │ AVERAGE    │
│ Brazil    │ 2018-04-30 │ AVERAGE    │
│ Brazil    │ 2018-05-01 │ AVERAGE    │
│ Brazil    │ 2018-05-02 │ LOW        │
│ Brazil    

In [45]:
process("""
SELECT
    c.country,
    w.date,
    AVG(w.temperature_c) as temperature_c,
    COALESCE(AVG(w.precipitation_mm), 0) as precipitation_mm
FROM (
    SELECT
        station_id,
        date::DATE as date,
        avg_temp_c as temperature_c,
        precipitation_mm,
    FROM '../../data/daily_weather.parquet'
    WHERE date BETWEEN '2017-01-01' AND '2021-12-31'
) AS w
JOIN '../../data/cities.csv' AS c
    ON w.station_id = c.station_id
GROUP BY c.country, w.date
HAVING temperature_c IS NOT NULL
ORDER BY c.country, w.date
""", "daily_country_weather")

daily_country_weather - 2
┌─────────────┬────────────┬──────────────────────┬──────────────────┐
│   country   │    date    │    temperature_c     │ precipitation_mm │
│   varchar   │    date    │        double        │      double      │
├─────────────┼────────────┼──────────────────────┼──────────────────┤
│ Afghanistan │ 2017-01-01 │   5.3166666666666655 │              0.0 │
│ Afghanistan │ 2017-01-02 │    5.016666666666667 │              2.0 │
│ Afghanistan │ 2017-01-03 │   3.0666666666666664 │           10.725 │
│ Afghanistan │ 2017-01-04 │                 2.65 │            109.0 │
│ Afghanistan │ 2017-01-05 │   1.9333333333333333 │            29.95 │
│ Afghanistan │ 2017-01-06 │   0.9833333333333331 │             7.25 │
│ Afghanistan │ 2017-01-07 │   0.2833333333333334 │             33.0 │
│ Afghanistan │ 2017-01-08 │ -0.07999999999999999 │             18.0 │
│ Afghanistan │ 2017-01-09 │ -0.21666666666666665 │              0.0 │
│ Afghanistan │ 2017-01-10 │    0.866666666666667 │

In [78]:
process("""
WITH wdi AS (
    SELECT
        "Country Name" as country,
        "Indicator Code" as code,
        "2016", "2017", "2018", "2019", "2020", "2021"
    FROM read_csv('../../data/WDIData.csv', header=True)
    WHERE code IN ('SP.RUR.TOTL.ZS', 'SP.DYN.TFRT.IN', 'NY.GDP.PCAP.CD', 'IT.CEL.SETS.P2', 'SM.POP.REFG', 'SP.POP.TOTL')
)
SELECT
    country,
    year,
    rural_population_percent,
    fertility_rate,
    gdp_per_capita_usd,
    mobile_subscriptions_per_100,
    refugee_population / total_population * 1000 AS refugee_population_promille,
FROM (
    SELECT
        country, year,
        MAX(CASE WHEN code = 'SP.RUR.TOTL.ZS' THEN value END) AS rural_population_percent,
        MAX(CASE WHEN code = 'SP.DYN.TFRT.IN' THEN value END) AS fertility_rate,
        MAX(CASE WHEN code = 'NY.GDP.PCAP.CD' THEN value END) AS gdp_per_capita_usd,
        MAX(CASE WHEN code = 'IT.CEL.SETS.P2' THEN value END) AS mobile_subscriptions_per_100,
        MAX(CASE WHEN code = 'SM.POP.REFG' THEN value END) AS refugee_population,
        MAX(CASE WHEN code = 'SP.POP.TOTL' THEN value END) AS total_population
    FROM (
        SELECT country, code, 2016 AS year, "2017" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2017 AS year, "2017" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2018 AS year, "2018" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2019 AS year, "2019" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2020 AS year, "2020" AS value FROM wdi
        UNION ALL
        SELECT country, code, 2021 AS year, "2021" AS value FROM wdi
    )
    GROUP BY country, year
)
WHERE rural_population_percent IS NOT NULL
    AND fertility_rate IS NOT NULL
    AND gdp_per_capita_usd IS NOT NULL
    AND mobile_subscriptions_per_100 IS NOT NULL
    AND refugee_population IS NOT NULL
    AND total_population IS NOT NULL
ORDER BY country, year
""", "wdi_normalized")

wdi_normalized - 0
┌─────────────────────────────┬───────┬──────────────────────────┬──────────────────┬────────────────────┬──────────────────────────────┬─────────────────────────────┐
│           country           │ year  │ rural_population_percent │  fertility_rate  │ gdp_per_capita_usd │ mobile_subscriptions_per_100 │ refugee_population_promille │
│           varchar           │ int32 │          double          │      double      │       double       │            double            │           double            │
├─────────────────────────────┼───────┼──────────────────────────┼──────────────────┼────────────────────┼──────────────────────────────┼─────────────────────────────┤
│ Afghanistan                 │  2016 │                    74.75 │            5.129 │   530.149830802984 │                  67.13641492 │           2.130182913434396 │
│ Afghanistan                 │  2017 │                    74.75 │            5.129 │   530.149830802984 │                  67.13641492 │    

In [93]:
process("""
SELECT
    prv.country,
    d.date,
    prv.rural_population_percent * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.rural_population_percent * (EXTRACT('dayofyear' FROM d.date) / 365) AS rural_population_percent,
    prv.fertility_rate * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.fertility_rate * (EXTRACT('dayofyear' FROM d.date) / 365) AS fertility_rate,
    prv.gdp_per_capita_usd * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.gdp_per_capita_usd * (EXTRACT('dayofyear' FROM d.date) / 365) AS gdp_per_capita_usd,
    prv.mobile_subscriptions_per_100 * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.mobile_subscriptions_per_100 * (EXTRACT('dayofyear' FROM d.date) / 365) AS mobile_subscriptions_per_100,
    prv.refugee_population_promille * (1 - (EXTRACT('dayofyear' FROM d.date) / 365)) + nxt.refugee_population_promille * (EXTRACT('dayofyear' FROM d.date) / 365) AS refugee_population_promille
FROM (
    SELECT generate_series::DATE AS date
    FROM generate_series(DATE '2017-01-01', DATE '2021-12-31', INTERVAL '1' DAY)
) AS d
JOIN './tmp/wdi_normalized.parquet' AS nxt
    ON YEAR(d.date) = nxt.year
JOIN './tmp/wdi_normalized.parquet' AS prv
    ON YEAR(d.date) - 1 = prv.year AND prv.country = nxt.country
ORDER BY prv.country, d.date
""", "wdi_interpolated")

wdi_interpolated - 12
┌─────────────┬────────────┬──────────────────────────┬────────────────────┬────────────────────┬──────────────────────────────┬─────────────────────────────┐
│   country   │    date    │ rural_population_percent │   fertility_rate   │ gdp_per_capita_usd │ mobile_subscriptions_per_100 │ refugee_population_promille │
│   varchar   │    date    │          double          │       double       │       double       │            double            │           double            │
├─────────────┼────────────┼──────────────────────────┼────────────────────┼────────────────────┼──────────────────────────────┼─────────────────────────────┤
│ Afghanistan │ 2017-01-01 │                    74.75 │              5.129 │   530.149830802984 │            67.13641492000002 │           2.130182913434396 │
│ Afghanistan │ 2017-01-02 │                    74.75 │              5.129 │  530.1498308029841 │                  67.13641492 │           2.130182913434396 │
│ Afghanistan │ 2017-01-